# Joint Bert Model for slot and intent classification

### Imports

In [59]:
import json
import os
import re
import time
import gpustat
import psutil
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense, Flatten, Reshape
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, CategoricalAccuracy, BinaryAccuracy, Precision, Recall, AUC

model_name = "bert-base-uncased"

# connect MLFlow
import mlflow
mlflow.login()

# set the experiment id
mlflow.set_experiment(experiment_id="3202139130034062")

mlflow.tensorflow.autolog(log_models=True, log_input_examples=False, log_model_signatures=False)

### Load Dataset

In [60]:
inputs = []
intentOutputs = []
slotOutputs = []

with open("../processing/JERTmate_final_data.json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

    inputs = data["inputs"]
    intentOutputs = data["intentOutputs"]
    slotOutputs = data["slotOutputs"]

### Split Data - Train 80% | Validation 10% | Test 10%

In [ ]:
def split_arrays(inputs, intentOutputs, slotOutputs, train_ratio, val_ratio, test_ratio):
    assert len(inputs) == len(intentOutputs) == len(slotOutputs), "All arrays must have the same length"
    
    n_total = len(inputs)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    
    # split inputs
    inputs_train, inputs_val, inputs_test = inputs[:n_train], inputs[n_train:n_train + n_val], inputs[n_train + n_val:]

    # split intents
    intentOutputs_train, intentOutputs_val, intentOutputs_test = intentOutputs[:n_train], intentOutputs[n_train:n_train + n_val], intentOutputs[n_train + n_val:]

    # split slots
    slot_type_map_train, slot_type_map_val, slot_type_map_test = [x[:50] for x in slotOutputs[:n_train]], [x[:50] for x in slotOutputs[n_train:n_train + n_val]], [x[:50] for x in slotOutputs[n_train + n_val:]]
    slot_intent_map_train, slot_intent_map_val, slot_intent_map_test = [x[50:100] for x in slotOutputs[:n_train]], [x[50:100] for x in slotOutputs[n_train:n_train + n_val]], [x[50:100] for x in slotOutputs[n_train + n_val:]]
    slot_action_map_train, slot_action_map_val, slot_action_map_test = [x[100:150] for x in slotOutputs[:n_train]], [x[100:150] for x in slotOutputs[n_train:n_train + n_val]], [x[100:150] for x in slotOutputs[n_train + n_val:]]
    slot_pointers_map_train, slot_pointers_map_val, slot_pointers_map_test = [x[150:300] for x in slotOutputs[:n_train]], [x[150:300] for x in slotOutputs[n_train:n_train + n_val]], [x[150:300] for x in slotOutputs[n_train + n_val:]]
    phantom_target_map_train, phantom_target_map_val, phantom_target_map_test = [x[300:305] for x in slotOutputs[:n_train]], [x[300:305] for x in slotOutputs[n_train:n_train + n_val]], [x[300:305] for x in slotOutputs[n_train + n_val:]]
    phantom_intent_map_train, phantom_intent_map_val, phantom_intent_map_test = [x[305:310] for x in slotOutputs[:n_train]], [x[305:310] for x in slotOutputs[n_train:n_train + n_val]], [x[305:310] for x in slotOutputs[n_train + n_val:]]
    phantom_action_map_train, phantom_action_map_val, phantom_action_map_test = [x[310:315] for x in slotOutputs[:n_train]], [x[310:315] for x in slotOutputs[n_train:n_train + n_val]], [x[310:315] for x in slotOutputs[n_train + n_val:]]
    phantom_pointers_map_train, phantom_pointers_map_val, phantom_pointers_map_test = [x[315:] for x in slotOutputs[:n_train]], [x[315:] for x in slotOutputs[n_train:n_train + n_val]], [x[315:] for x in slotOutputs[n_train + n_val:]]

    
    return (tf.constant(inputs_train), tf.constant(inputs_val), tf.constant(inputs_test)), (tf.constant(intentOutputs_train), tf.constant(intentOutputs_val), tf.constant(intentOutputs_test)), (tf.constant(slot_type_map_train), tf.constant(slot_type_map_val), tf.constant(slot_type_map_test)), (tf.constant(slot_intent_map_train), tf.constant(slot_intent_map_val), tf.constant(slot_intent_map_test)), (tf.constant(slot_action_map_train), tf.constant(slot_action_map_val), tf.constant(slot_action_map_test)), (tf.constant(slot_pointers_map_train), tf.constant(slot_pointers_map_val), tf.constant(slot_pointers_map_test)), (tf.constant(phantom_target_map_train), tf.constant(phantom_target_map_val), tf.constant(phantom_target_map_test)), (tf.constant(phantom_intent_map_train), tf.constant(phantom_intent_map_val), tf.constant(phantom_intent_map_test)), (tf.constant(phantom_action_map_train), tf.constant(phantom_action_map_val), tf.constant(phantom_action_map_test)), (tf.constant(phantom_pointers_map_train), tf.constant(phantom_pointers_map_val), tf.constant(phantom_pointers_map_test))


train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

print(len(inputs))

(inputs_train, inputs_val, inputs_test), (intentOutputs_train, intentOutputs_val, intentOutputs_test), (slot_type_map_train, slot_type_map_val, slot_type_map_test), (slot_intent_map_train, slot_intent_map_val, slot_intent_map_test), (slot_action_map_train, slot_action_map_val, slot_action_map_test), (slot_pointers_map_train, slot_pointers_map_val, slot_pointers_map_test), (phantom_target_map_train, phantom_target_map_val, phantom_target_map_test), (phantom_intent_map_train, phantom_intent_map_val, phantom_intent_map_test), (phantom_action_map_train, phantom_action_map_val, phantom_action_map_test), (phantom_pointers_map_train, phantom_pointers_map_val, phantom_pointers_map_test) = split_arrays(inputs, intentOutputs, slotOutputs, train_ratio, val_ratio, test_ratio)

print(intentOutputs_train.shape)

### Define Model

In [ ]:
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_vector_length=None, total_slot_number=None, total_phantom_slot_number=None, slot_types=None, slot_intents=None, pointer_possibilities=None, model_name=model_name, dropout_prob=0.05):
        super().__init__(name="joint_intent_slot")
        #   ** GENERAL LAYERS **
        self.bert = TFBertModel.from_pretrained(model_name) # BERT model
        self.dropout = Dropout(dropout_prob) # basic dropout layer
        self.flatten = Flatten() # flatten layer



        #   ** SLOT LAYERS **
        # LHS compressor
        #?self.conv_layer = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 768), strides=(1, 1),padding='valid',activation='relu')
        self.last_hidden_sequence_compressor = Dense(768, activation="relu", name="last_hidden_sequence_compressor")

        # processing layers
        #?self.slot_processor = Dense(((150) + (intent_vector_length * 2) + (slot_vector_length * 2)), activation="relu", name="slot_processor")

        # slot output layers
        self.slot_type_dense = Dense(total_slot_number * slot_types, activation='softmax', name="slot_type_output")
        self.slot_type_reshape = Reshape((total_slot_number, slot_types))
        
        self.slot_intent_dense = Dense(total_slot_number * slot_intents, activation='softmax', name="slot_intent_output")
        self.slot_intent_reshape = Reshape((total_slot_number, slot_intents))
        
        self.slot_action_output = Dense(total_slot_number, activation='sigmoid', name="slot_action_output")
        
        self.slot_pointers_dense = Dense(total_slot_number * pointer_possibilities * 3, activation='softmax', name="slot_pointers_output")
        self.slot_pointers_reshape = Reshape((total_slot_number * 3, pointer_possibilities))

        # Phantom slot output layers
        self.phantom_slot_target_dense = Dense(total_phantom_slot_number * pointer_possibilities, activation='softmax', name="phantom_slot_target_output")
        self.phantom_slot_target_reshape = Reshape((total_phantom_slot_number, pointer_possibilities))
        
        self.phantom_slot_intent_dense = Dense(total_phantom_slot_number * slot_intents, activation='softmax', name="phantom_slot_intent_output")
        self.phantom_slot_intent_reshape = Reshape((total_phantom_slot_number, slot_intents))
        
        self.phantom_slot_action_output = Dense(total_phantom_slot_number, activation='sigmoid', name="phantom_slot_action_output")
        
        self.phantom_slot_pointers_dense = Dense(total_phantom_slot_number * pointer_possibilities * 3, activation='softmax', name="phantom_slot_pointers_output")
        self.phantom_slot_pointers_reshape = Reshape((total_phantom_slot_number * 3, pointer_possibilities))



        #  ** INTENT LAYERS **
        # processing layers
        #?self.intent_processor = Dense(((150 * 1) + (intent_vector_length * 2)), activation="relu", name="intent_processor")

        # output layer
        self.intent_output = Dense(intent_vector_length, activation='softmax', name="intent_output")

        # Build the model with input shape (None, 886)
        self.build(input_shape=(None, 886))

    def call(self, inputs, **kwargs):
        bertInputs = inputs[:, :150]

        # run BERT
        trained_bert = self.bert(bertInputs, **kwargs)
        pooled_output = trained_bert.pooler_output
        sequence_output = trained_bert.last_hidden_state

        #   ** SLOT CLASSIFICATION **
        # reduce dimensionality of sequence output - Dense(150)
        flattened_sequence_output = self.flatten(sequence_output)
        LHSC_output = self.last_hidden_sequence_compressor(flattened_sequence_output)

        # slot processor
        #slot_processor_input = self.dropout(tf.concat([LHSC_output, tf.cast(inputs[:, 150:], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        #slot_processor_output = self.slot_processor(slot_processor_input)

        # slot output
        slot_output_input = self.dropout(tf.concat([LHSC_output, tf.cast(inputs[:, 150:], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        
        slot_type_output = self.slot_type_dense(slot_output_input)
        slot_type_output = self.slot_type_reshape(slot_type_output)
        
        slot_intent_output = self.slot_intent_dense(slot_output_input)
        slot_intent_output = self.slot_intent_reshape(slot_intent_output)
        
        slot_action_output = self.slot_action_output(slot_output_input)
        
        slot_pointers_output = self.slot_pointers_dense(slot_output_input)
        slot_pointers_output = self.slot_pointers_reshape(slot_pointers_output)

        # Phantom slot outputs
        phantom_target_output = self.phantom_slot_target_dense(slot_output_input)
        phantom_target_output = self.phantom_slot_target_reshape(phantom_target_output)
        
        phantom_intent_output = self.phantom_slot_intent_dense(slot_output_input)
        phantom_intent_output = self.phantom_slot_intent_reshape(phantom_intent_output)
        
        phantom_action_output = self.phantom_slot_action_output(slot_output_input)
        
        phantom_pointers_output = self.phantom_slot_pointers_dense(slot_output_input)
        phantom_pointers_output = self.phantom_slot_pointers_reshape(phantom_pointers_output)



        #   ** INTENT CLASSIFICATION **
        # intent processor
        #intent_processor_input = self.dropout(tf.concat([pooled_output, tf.cast(inputs[:, 150:150 + 114], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        #intent_processor_output = self.intent_processor(intent_processor_input)

        # intent output
        intent_output_input = self.dropout(tf.concat([pooled_output, tf.cast(inputs[:, 150:150 + 114], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        intent_output = self.intent_output(intent_output_input)

        # Return outputs as a dictionary
        return {
            "intent_output": intent_output,
            "slot_type_output": slot_type_output,
            "slot_intent_output": slot_intent_output,
            "slot_action_output": slot_action_output,
            "slot_pointers_output": slot_pointers_output,
            "phantom_slot_target_output": phantom_target_output,
            "phantom_slot_intent_output": phantom_intent_output,
            "phantom_slot_action_output": phantom_action_output,
            "phantom_slot_pointers_output": phantom_pointers_output
        }

    def get_config(self):
        config = super(JointIntentAndSlotFillingModel, self).get_config()
        return config
    
    def build(self, input_shape):
        super().build(input_shape)
        self.input_shape = input_shape

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
joint_model = JointIntentAndSlotFillingModel(intent_vector_length=38, total_slot_number=50, total_phantom_slot_number=5, slot_types=15, slot_intents=4, pointer_possibilities=18)

### Train and Compile

In [ ]:
# optimizer
opt = AdamW(learning_rate=1e-3, weight_decay=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

# learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 15:
        return lr
    else:
        return lr * 0.97
learning_rate_callback = LearningRateScheduler(scheduler)

# Model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath="model_epoch_{epoch:02d}.keras",  # Save the model with the epoch number in the filename
    save_freq='epoch',
    verbose=1 
)

# Run time logging callback
class PredictionTimeLoggingCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data):
        super(PredictionTimeLoggingCallback, self).__init__()
        self.validation_data = validation_data
    
    def on_epoch_end(self, epoch, logs=None):
        # Measure the time it takes to make predictions
        start_time = time.time()
        
        # Predict on the validation data
        _ = self.model.predict(self.validation_data[0])
        
        end_time = time.time()
        prediction_time = end_time - start_time

        mlflow.log_metric("prediction_time", prediction_time, step=epoch)
        
        print(f"Prediction time for epoch {epoch}: {prediction_time:.4f} seconds")

prediction_time_callback = PredictionTimeLoggingCallback(inputs_val)

# computing power logging callback
class ResourceMonitoringCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Monitor GPU usage
        gpu_stats = gpustat.GPUStatCollection.new_query()
        gpu_memory_usage = gpu_stats[0].memory_used
        gpu_utilization = gpu_stats[0].utilization

        # Monitor CPU memory usage
        memory_info = psutil.virtual_memory()
        cpu_memory_usage = memory_info.used / (1024 ** 3)  # Convert to GB

        # Log metrics to MLFlow
        mlflow.log_metric("gpu_memory_usage", gpu_memory_usage, step=epoch)
        mlflow.log_metric("gpu_utilization", gpu_utilization, step=epoch)
        mlflow.log_metric("cpu_memory_usage", cpu_memory_usage, step=epoch)

        print(f"Epoch {epoch}: GPU Memory Usage: {gpu_memory_usage} MB, GPU Utilization: {gpu_utilization}%, CPU Memory Usage: {cpu_memory_usage} GB")

resource_monitoring_callback = ResourceMonitoringCallback()

# loss functions
losses = {
    "intent_output": CategoricalCrossentropy(name="intent_loss"), 
    "slot_type_output": SparseCategoricalCrossentropy(from_logits=True, name="slot_type_loss"),
    "slot_intent_output": SparseCategoricalCrossentropy(from_logits=True, name="slot_intent_loss"),
    "slot_action_output": BinaryCrossentropy(name="slot_actionable_loss"), 
    "slot_pointers_output": SparseCategoricalCrossentropy(from_logits=True, name="slot_pointer_loss"),
    "phantom_slot_target_output": SparseCategoricalCrossentropy(from_logits=True, name="phantom_slot_target_loss"),
    "phantom_slot_intent_output": SparseCategoricalCrossentropy(from_logits=True, name="phantom_slot_intent_loss"),
    "phantom_slot_action_output": BinaryCrossentropy(name="phantom_slot_actionable_loss"), 
    "phantom_slot_pointers_output": SparseCategoricalCrossentropy(from_logits=True, name="phantom_slot_pointer_loss"),
}

# loss weights
loss_weights = {
    "intent_output": 1.5,
    "slot_type_output": 1.0,
    "slot_intent_output": 1.0,
    "slot_action_output": 0.6,
    "slot_pointers_output": 1.0,
    "phantom_slot_target_output": 0.8,
    "phantom_slot_intent_output": 0.8,
    "phantom_slot_action_output": 0.6,
    "phantom_slot_pointers_output": 0.8,
}

# metrics
metrics = {
    "intent_output": [
        CategoricalAccuracy(name="intent_accuracy"), 
    ],
    "slot_type_output": [
        SparseCategoricalAccuracy(name="slot_type_accuracy"), 
    ],
    "slot_intent_output": [
        SparseCategoricalAccuracy(name="slot_intent_accuracy"), 
    ],
    "slot_action_output": [
        BinaryAccuracy(name="slot_action_accuracy"), 
    ],
    "slot_pointers_output": [
        SparseCategoricalAccuracy(name="slot_pointer_accuracy"), 
    ],
    "phantom_slot_target_output": [
        SparseCategoricalAccuracy(name="phantom_slot_target_accuracy"), 
    ],
    "phantom_slot_intent_output": [
        SparseCategoricalAccuracy(name="phantom_slot_intent_accuracy"), 
    ],
    "phantom_slot_action_output": [
        BinaryAccuracy(name="phantom_slot_action_accuracy"), 
    ],
    "phantom_slot_pointers_output": [
        SparseCategoricalAccuracy(name="phantom_slot_pointer_accuracy"), 
    ]
}

# compile model
joint_model.compile(optimizer=opt, loss=losses, loss_weights=loss_weights, metrics=metrics)

# train!
history = joint_model.fit(
    x=inputs_train,
    y={
        "intent_output": intentOutputs_train,
        "slot_type_output": slot_type_map_train,
        "slot_intent_output": slot_intent_map_train,
        "slot_action_output": slot_action_map_train,
        "slot_pointers_output": slot_pointers_map_train,
        "phantom_slot_target_output": phantom_target_map_train,
        "phantom_slot_intent_output": phantom_intent_map_train,
        "phantom_slot_action_output": phantom_action_map_train,
        "phantom_slot_pointers_output": phantom_pointers_map_train
    }, 
    validation_data=(
        inputs_val,
        {
            "intent_output": intentOutputs_val,
            "slot_type_output": slot_type_map_val,
            "slot_intent_output": slot_intent_map_val,
            "slot_action_output": slot_action_map_val,
            "slot_pointers_output": slot_pointers_map_val,
            "phantom_slot_target_output": phantom_target_map_val,
            "phantom_slot_intent_output": phantom_intent_map_val,
            "phantom_slot_action_output": phantom_action_map_val,
            "phantom_slot_pointers_output": phantom_pointers_map_val
        }
    ),
    epochs=24,
    batch_size=22,
    shuffle=True,
    callbacks=[learning_rate_callback, checkpoint_callback, resource_monitoring_callback, prediction_time_callback],
)

### Evaluate Model

keep in mind -> test slot accuracy will be higher than reality.
        Because 90% of the data points are 0, it can just guess 0 and be right 85% of the time

In [ ]:
test_loss, test_intent_acc, test_slot_type, test_slot_intent, test_slot_action, test_slot_pointers, test_phantom_target, test_phantom_intent, test_phantom_action, test_phantom_pointers = joint_model.evaluate(x=inputs_test, y=(intentOutputs_test, slot_type_map_test, slot_intent_map_test, slot_action_map_test, slot_pointers_map_test, phantom_target_map_test, phantom_intent_map_test, phantom_action_map_test, phantom_pointers_map_test), batch_size=22)

print(f"Test Intent Accuracy: {test_intent_acc}")
print(f"Test Slot Type Accuracy: {test_slot_type}")
print(f"Test Slot Intent Accuracy: {test_slot_intent}")
print(f"Test Slot Action Accuracy: {test_slot_action}")
print(f"Test Slot Pointers Accuracy: {test_slot_pointers}")
print(f"Test Phantom Target Accuracy: {test_phantom_target}")
print(f"Test Phantom Intent Accuracy: {test_phantom_intent}")
print(f"Test Phantom Action Accuracy: {test_phantom_action}")
print(f"Test Phantom Pointers Accuracy: {test_phantom_pointers}")

### Inference

In [ ]:
def inferConversation():
    

def inferSentence():

